We initialize the camera

In [2]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

RuntimeError: Could not initialize camera.  Please see error trace.

We load the model

In [8]:
import torch
import torchvision
from torch2trt import TRTModule

#TRT
model = TRTModule()
model.load_state_dict(torch.load('road_following_alexnet50_trt.pth'))

#ALEXNET
#model = torchvision.models.alexnet(pretrained=False)
#model.classifier[-1] = torch.nn.Linear(4096, 2)
#model = model.cuda().eval().half()
#model.load_state_dict(torch.load('road_following_alexnet50.pth'))


device = torch.device('cuda')
model = model.to(device)
# SQUEEZENET 
#model = torchvision.models.squeezenet1_1(pretrained=False)
#model.classifier[1] = torch.nn.Conv2d(512, 2, kernel_size=1)
#model.num_classes = 1
#model = model.cuda().eval().half()
#model.load_state_dict(torch.load('road_following_squeeze50.pth'))

We initialize the car and import the preprocess function

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar
from utils import preprocess
import numpy as np

car = NvidiaRacecar()


We define the interface

In [6]:
from IPython.display import display
import ipywidgets.widgets as widgets


fps = widgets.FloatText(
    value=0.0,
    description='FPS:',
    disabled=False
)


steering_disp = widgets.FloatProgress(
    value=0.0,
    min=-1.0,
    max=1.0,
    description='Direction:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)





Main function

In [7]:
from utils import preprocess
from datetime import datetime
display(fps,steering_disp)
import time
t_tot_1 = time.time() 

#Set the gain and bias
STEERING_GAIN = 1
STEERING_BIAS = 0.25


#Delete buffer
for i in range(50):
    image = camera.read()
car.throttle = -0.0
#initialize time
t_tot_1 = time.time()

#open file to save FPS
f = open("live_alexnetB.txt", "a")
f.write(str(datetime.now())+"\n")
f.write("fps fps(tot) time \n")

#we initialize timers and counters
n = 0
m = 0
time_sum = 0.0
t_1 = time.time()


while True:
    #read and process image
    for i in range(3):
        image = camera.read()
    image = preprocess(image).half()
    t = time.time()
    output = model(image).detach().cpu().numpy().flatten()
    time_sum  += time.time()-t
    x = float(output[0])
    
    #Set steering
    steering_disp.value = x * STEERING_GAIN + STEERING_BIAS
    car.steering = x * STEERING_GAIN + STEERING_BIAS
    
    #Calculate and write FPS each 10 cycles, leave while after 100 cycles
    n += 1
    if n == 10:
        fps_tot = 10/(time.time()-t_1)
        fps_com = 10/(time_sum)
        fps.value = fps_tot
        n = 0
        m += 1
        f.write(str(fps_com)+" "+str(fps_tot)+" "+str(time.time()-t_0)+"\n")
        time_sum =0
        t_1 = time.time()
        if m == 100:
            f.close()
            break

#stop the car           
car.throttle = 0
    

FloatText(value=0.0, description='FPS:')

FloatText(value=0.0, description='X:')

FloatProgress(value=0.0, bar_style='info', description='Direction:', max=1.0, min=-1.0, style=ProgressStyle(ba…

FloatText(value=0.0, description='Steering gain:')

KeyboardInterrupt: 